In [ ]:
# First attempt at an interpreter
# We will still find the G84 and index their line numbers
# But we will do a word by word interpretation after
# "Words" are commands seperated by spaces
# Every word will have a space or newline following it, OR will start with a significant character like G,M,T,H,D etc
# We will define classes for the following:
#       Reading a line and putting it into an array of words
#       Reading an array of words for certain key words and their synonyms (M6 = M06)
#           Sepereate letter codes (M00006 = M code #6)

In [2]:
# Open file in read mode
from rich import print

filePath = "Sample Gcode/Solidworks/CGIP11719.NC"

with open(filePath, 'r') as file:
    lines = file.readlines()

In [48]:
# Separate one line into individual keywords
testLine = lines[807]
print("Original:",testLine)
# List of letters words start with:

def readLine(line):
    codeList = ['G','M','T','S','F','X','Y','Z','H','D','N','R','P']
    newWord = ""
    wordList = []
    for i, char in enumerate(line):
        if char in codeList or char == "\n" or char == " ":
            if newWord != "" and newWord != " ":
                wordList.append(newWord)
            newWord = ""
        newWord += char
    return(wordList)

testResult = readLine(testLine)
print("Word list:", testResult)

In [49]:
# Now we need to read a word list and interpret information. The file has to be read upwards from the line containing the G84 to use the most recent feeds, speeds, and tool numbers

# Set all values to false every time they are reset
valueDict = {"feed":False, "rpm":False, "tool":False}

def readWords(words, values):
    for word in words:
        letter = word[0] # Separate key letter from value
        value = word[1:]

        if letter == 'F':
            if values["feed"] == False:
                values["feed"] = value
        elif letter == 'S':
            if values["rpm"]== False:
                values["rpm"] = value
        elif letter == 'T':
            if values["tool"] == False:
                values["tool"] = value
        # print((letter,value))
    return(values)



In [50]:
# Full program idea:
# Find all G84 lines
# Read G84 line and up to tool change
# List all stats for the tool in a dict

tappingCycleIndex = []
for lineIndex, line in enumerate(lines):
    if line.find("G84") != -1:
        tappingCycleIndex.append(lineIndex+1)

allValues = []

for i, tappingCycle in enumerate(tappingCycleIndex):
    valueDict = {"feed":False, "rpm":False, "tool":False}
    lineToRead = tappingCycle
    while False in valueDict.values():
        words = readLine(lines[lineToRead])
        readWords(words, valueDict)
        lineToRead -= 1
    allValues.append(valueDict)

print(allValues)
